In [1]:
import pandas as pd
import sys  
sys.path.insert(1, '/home/tb24/projects/llm-data-aug')

# Path
import os

# Get the project root directory (one level up from the notebook directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define the data path
data_path = os.path.join(project_root, "data")

## Examining Class Imbalance

In [2]:
# Read the cleaned data
df = pd.read_csv(os.path.join(data_path, "cleaned_user_reviews.csv"))
# Make sure they are all of type string
df['emoji to text'] = df['emoji to text'].astype(str)
df

,Review,Sentiment,tokenized_text,emoji to text,emoji to text + stopwords
0,rất cơ bản.,Positive,rất cơ_bản .,rất cơ_bản .,cơ_bản
1,họ đã miễn phí thịt xông khói thịt lợn trong s...,Neutral,họ đã miễn_phí thịt xông khói thịt lợn trong s...,họ đã miễn_phí thịt xông khói thịt lợn trong s...,miễn_phí thịt xông khói thịt lợn salad hồi_giá...
2,"trong mcdonalds, họ cho tóc ăn.",Negative,"trong mcdonalds , họ cho tóc ăn .","trong mcdonalds , họ cho tóc ăn .",mcdonalds tóc
3,khó đặt hàng,Positive,khó đặt_hàng,khó đặt_hàng,đặt_hàng
4,"đẳng cấp hơn các loại gà rán khác, cảm giác đa...",Positive,"đẳng_cấp hơn các loại gà rán khác , cảm_giác đ...","đẳng_cấp hơn các loại gà rán khác , cảm_giác đ...",đẳng_cấp gà rán mỹ vn 2200 đông ..
...,...,...,...,...,...
2373,buổi trưa ngồi bị hắt nắng,Positive,buổi trưa ngồi bị hắt nắng,buổi trưa ngồi bị hắt nắng,trưa hắt nắng
2374,tôi đặt hàng từ mcdonalds khá thường xuyên sử ...,Neutral,tôi đặt_hàng từ mcdonalds khá thường_xuyên sử_...,tôi đặt_hàng từ mcdonalds khá thường_xuyên sử_...,đặt_hàng mcdonalds thường_xuyên sử_dụng ứng_dụ...
2375,ok... điểm hẹn lý tưởng cuối tuần cho các bé,Positive,ok ... điểm hẹn lý_tưởng cuối tuần cho các bé,ok ... điểm hẹn lý_tưởng cuối tuần cho các bé,ok ... hẹn lý_tưởng bé
2376,tuyệt vời khi đi bạn bè đông,Positive,tuyệt_vời khi đi bạn_bè đông,tuyệt_vời khi đi bạn_bè đông,tuyệt_vời bạn_bè đông


In [3]:
# Định nghĩa mapping
label_mapping = {'Positive': 1, 'Neutral': 2, 'Negative': 0}

# Chuyển đổi cột 'Sentiment' sang số bằng cách sử dụng mapping
df['Sentiment'] = df['Sentiment'].map(label_mapping)

df

,Review,Sentiment,tokenized_text,emoji to text,emoji to text + stopwords
0,rất cơ bản.,1,rất cơ_bản .,rất cơ_bản .,cơ_bản
1,họ đã miễn phí thịt xông khói thịt lợn trong s...,2,họ đã miễn_phí thịt xông khói thịt lợn trong s...,họ đã miễn_phí thịt xông khói thịt lợn trong s...,miễn_phí thịt xông khói thịt lợn salad hồi_giá...
2,"trong mcdonalds, họ cho tóc ăn.",0,"trong mcdonalds , họ cho tóc ăn .","trong mcdonalds , họ cho tóc ăn .",mcdonalds tóc
3,khó đặt hàng,1,khó đặt_hàng,khó đặt_hàng,đặt_hàng
4,"đẳng cấp hơn các loại gà rán khác, cảm giác đa...",1,"đẳng_cấp hơn các loại gà rán khác , cảm_giác đ...","đẳng_cấp hơn các loại gà rán khác , cảm_giác đ...",đẳng_cấp gà rán mỹ vn 2200 đông ..
...,...,...,...,...,...
2373,buổi trưa ngồi bị hắt nắng,1,buổi trưa ngồi bị hắt nắng,buổi trưa ngồi bị hắt nắng,trưa hắt nắng
2374,tôi đặt hàng từ mcdonalds khá thường xuyên sử ...,2,tôi đặt_hàng từ mcdonalds khá thường_xuyên sử_...,tôi đặt_hàng từ mcdonalds khá thường_xuyên sử_...,đặt_hàng mcdonalds thường_xuyên sử_dụng ứng_dụ...
2375,ok... điểm hẹn lý tưởng cuối tuần cho các bé,1,ok ... điểm hẹn lý_tưởng cuối tuần cho các bé,ok ... điểm hẹn lý_tưởng cuối tuần cho các bé,ok ... hẹn lý_tưởng bé
2376,tuyệt vời khi đi bạn bè đông,1,tuyệt_vời khi đi bạn_bè đông,tuyệt_vời khi đi bạn_bè đông,tuyệt_vời bạn_bè đông


In [4]:
sentiment_counts = df['Sentiment'].value_counts()

print("Số lượng nhãn trong cột 'Sentiment':")
print(sentiment_counts)

Số lượng nhãn trong cột 'Sentiment':
Sentiment
1    1687
0     380
2     311
Name: count, dtype: int64


In [7]:
print(max(sentiment_counts))

1687


In [9]:
# Take out 15 examples for each class
df_positive = df[df['Sentiment'] == 1].sample(15)
df_negative = df[df['Sentiment'] == 0].sample(15)
df_neutral = df[df['Sentiment'] == 2].sample(15)

df_negative.head()

df_sampled = pd.concat([df_positive, df_negative, df_neutral])

df_sampled.to_csv(os.path.join(data_path, "class_samples/user_review_class_samples.csv"), index=False)

## Data Augmentation with Resampling

In [ ]:
from sklearn.utils import resample

# Create DataFrames for each sentiment class
df_positive = df[df['Sentiment'] == 1]
df_negative = df[df['Sentiment'] == 0]
df_neutral = df[df['Sentiment'] == 2]

# Get size of the majority class
n_samples = len(df_positive)

df_positive.head()

#### Upsampling

In [ ]:
# Upsample minority classes to match majority class
df_negative_upsampled = resample(
    df_negative, 
    replace=True,
    n_samples=n_samples,
    random_state=42
)

df_neutral_upsampled = resample(
    df_neutral,
    replace=True,
    n_samples=n_samples,
    random_state=42
)

# Combine upsampled minority classes with majority class
df_upsampled = pd.concat([df_positive, df_negative_upsampled, df_neutral_upsampled])

# Check the distribution of the upsampled dataset
print("Class distribution after upsampling:")
print(df_upsampled['Sentiment'].value_counts())

# Save the upsampled dataset if needed
df_upsampled.to_csv(os.path.join(data_path, "upsampled/upsampled_user_reviews.csv"), index=False)

In [ ]:
df_negative_upsampled

### Downsampling

In [ ]:
# Size of the minority classes
len(df_negative), len(df_neutral)

In [ ]:
# Downsample the majority class to match the minority class
min_samples = min(len(df_negative), len(df_neutral)) 

df_positive_downsampled = resample(
    df_positive,
    replace=False,
    n_samples=min_samples,
    random_state=42
)

df_negative_downsampled = resample(
    df_negative,
    replace=False,
    n_samples=min_samples,
    random_state=42
)


# Combine downsampled majority classes with minority class
df_downsampled = pd.concat([df_positive_downsampled, df_neutral, df_negative_downsampled])

# Check the distribution of the downsampled dataset
print("Class distribution after downsampling:")
print(df_downsampled['Sentiment'].value_counts())

# Save the downsampled dataset if needed
df_downsampled.to_csv(os.path.join(data_path, "downsampled/downsampled_user_reviews.csv"), index=False)


In [ ]:
df_downsampled.head()

## Split data into train, val, test

In [2]:
from dataloaders.train_test_split import DataScenario, TrainTestSplit

TrainTestSplit.run_train_test_split(DataScenario.UPSAMPLED)

In [4]:
TrainTestSplit.run_train_test_split(DataScenario.DOWNSAMPLED)